In [7]:
!pip install ipython-sql

In [29]:
import sqlite3
import pandas as pd
import backtrader as bt

In [30]:
df1 = pd.read_csv('data.csv')

In [25]:
df2 = df1.groupby('symbol')
for key, item in df2:
    print(df2.get_group(key), "\n\n")

       symbol localsymbol     osid  tradedate  previoustradedate   close
11421  HDS.IN    HDFCLIFE  3153738   20171117                NaN  344.60
11423  HDS.IN    HDFCLIFE  3153738   20171120         20171117.0  355.35
11429  HDS.IN    HDFCLIFE  3153738   20171121         20171120.0  385.30
11431  HDS.IN    HDFCLIFE  3153738   20171122         20171121.0  395.20
11437  HDS.IN    HDFCLIFE  3153738   20171123         20171122.0  374.85
...       ...         ...      ...        ...                ...     ...
15895  HDS.IN    HDFCLIFE  3153738   20220530         20220527.0  608.90
15901  HDS.IN    HDFCLIFE  3153738   20220531         20220530.0  598.95
15903  HDS.IN    HDFCLIFE  3153738   20220601         20220531.0  607.15
15909  HDS.IN    HDFCLIFE  3153738   20220602         20220601.0  607.90
15911  HDS.IN    HDFCLIFE  3153738   20220603         20220602.0  614.15

[1124 rows x 6 columns] 


       symbol localsymbol     osid  tradedate  previoustradedate  close
4526   ILC.IN        IDE

In [28]:
rel = df2.get_group('REL.IN')
ilc = df2.get_group('ILC.IN')
hds = df2.get_group('HDS.IN')
wip = df2.get_group('WIP.IN')

In [61]:
class TestStrategy(bt.Strategy):
    params = (
    ('fast', 50),
    ('slow', 200)
    )
    
    def __init__(self):
        self.crossovers = []
        
        for d in self.datas:
            ma_fast = bt.ind.SMA(d, period = self.params.fast)
            ma_slow = bt.ind.SMA(d, period = self.params.slow)
            
            self.crossovers.append(bt.ind.CrossOver(ma_fast, ma_slow))
            
    def next(self):
        for i,d in enumerate(self.datas):
            if not self.getposition(d).size:
                if self.crossovers[i] > 0:
                    self.buy(data = d)
                    
            elif self.crossovers[i] < 0:
                self.close(data = d)
            
            
        

In [43]:
class PandasData(bt.feed.DataBase):
    #linesoverride = True
    #lines = ('close')
    params = (
        ('datetime', 'tradedate'),
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', 'close'),
        ('volume', -1),
        ('openinterest', -1)
        )

In [53]:
cerebro = bt.Cerebro()

In [54]:
stocks = {'rel':rel, 'ilc':ilc, 'hds':hds, 'wip':wip}
for s in stocks:
    data = bt.feeds.PandasData(dataname = stocks[s])
    cerebro.adddata(data, name = s)

In [62]:
cerebro.addstrategy(TestStrategy)
cerebro.broker.setcash(100000000.0)
cerebro.run

<bound method Cerebro.run of <backtrader.cerebro.Cerebro object at 0x7fb0bf941520>>

In [63]:
cerebro.broker.getvalue()

100000000.0